In [22]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif = Image.open(filename)._getexif()

    if exif is not None:
        exifkeys = [x for x in exif.keys()]
        for key in exifkeys:
            name = TAGS.get(key,key)
            exif[name] = exif.pop(key)
            
        if 'GPSInfo' in exif:
            exifkeys2 = [x for x in exif['GPSInfo'].keys()]
            for key in exifkeys2:
                name = GPSTAGS.get(key,key)
                exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

In [23]:
import re
def get_coords(info):
    for key in ['Latitude','Longitude']:
        sign = -1 if info.get('GPS'+key+'Ref') in ['S','W'] else 1
        numbers = info.get('GPS'+key)
        degree = numbers[0]
        minute = numbers[1]
        second = numbers[2]
        info[key] = sign * (int(degree) + float(minute) / 60 + float(second) / 3600)
    return info['Latitude'], info['Longitude']

In [18]:
import pandas as pd
import os
data = pd.DataFrame(columns = ['Filename','Lat','Long','CD','BT'])
filenames = []
latitudes = [] 
longitudes = []
timestamps = []
for image in os.listdir('geotagged frames/'):
    exif = get_exif('geotagged frames/'+image)
    lat, long = get_coords(exif['GPSInfo'])
    filenames.append(image)
    latitudes.append(lat)
    longitudes.append(long)
data['Filename'] = filenames
data['Lat'] = latitudes
data['Long'] = longitudes

In [20]:
from pigeon import annotate
from IPython.display import display, Image

    
#annotations = annotate(
  #['assets/img_example1.jpg', 'assets/img_example2.jpg'],
  #options=['cat', 'dog', 'horse'],
  #display_fn=lambda filename: display(Image(filename))
#)

In [37]:
info = get_exif('geotagged frames/GH020013.MP4_60.jpg')
print(info['GPS'.get('GPSTimeStamp'))

None


In [33]:
time = info.get('GPSTimeStamp')
date = info.get('GPSDateStamp')
print(date)

None


In [25]:
timestr = ''
for x in range(len(time)):
    if x < len(time) - 1:
        timestr.append(str(time[x])+':')
    else:
        timestr.append(time[x])
print(timestr)

TypeError: object of type 'NoneType' has no len()